In [158]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [159]:
# Cargar el modelo guardado
modelo_cargado = tf.keras.models.load_model('modelo_entrenado_v1_binario.h5')

In [160]:
# Obtener la forma de la entrada esperada por el modelo
input_shape = modelo_cargado.input_shape
print(f"Input shape expected by the model: {input_shape}")

# También podemos inspeccionar un resumen completo del modelo para entender mejor su estructura
modelo_cargado.summary()

Input shape expected by the model: (None, 13)


Model: "sequential_49"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_196 (Dense)               │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_147 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_197 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_148 (Dropout)           │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_198 (Dense)               │ (None, 30)             │           990 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_149 (Dropout)           │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_199 (Dense)               │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,999 (15.62 KB)

 Trainable params: 3,997 (15.61 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [161]:
import joblib

# Cargar el scaler y encoder guardados
scaler = joblib.load('scaler.pkl')
enc = joblib.load('encoder.pkl')

In [162]:
# Preprocesar los nuevos datos (el mismo proceso que antes)
df = pd.DataFrame({
    'Soil_Type': ['loam', 'sandy'],
    'Sunlight_Hours': [7.5, 5.2],
    'Growth_Milestone':[1.3,5.2],
    'Water_Frequency': ['bi-weekly', 'daily'],
    'Fertilizer_Type': ['organic', 'none'],
    'Temperature': [22.0, 18.5],
    'Humidity': [55.0, 60.0]
})

In [163]:
df.head()

,Soil_Type,Sunlight_Hours,Growth_Milestone,Water_Frequency,Fertilizer_Type,Temperature,Humidity
0,loam,7.5,1.3,bi-weekly,organic,22.0,55.0
1,sandy,5.2,5.2,daily,none,18.5,60.0


In [164]:
# Definir características numéricas y categóricas

numeric_features = ['Sunlight_Hours','Temperature','Humidity']
categorical_features = ['Soil_Type','Water_Frequency','Fertilizer_Type']

In [165]:
# Aplicar el scaler a las características numéricas
df[numeric_features] = scaler.transform(df[numeric_features])

In [166]:
# One-Hot Encoding para características categóricas
encoded_cats = enc.transform(df[categorical_features])

In [167]:
# Crear DataFrame para las características codificadas
encoded_cats_df = pd.DataFrame(encoded_cats, columns=enc.get_feature_names_out(categorical_features))
df = pd.concat([df.drop(columns=categorical_features), encoded_cats_df], axis=1)

In [168]:
df.head()

,Sunlight_Hours,Growth_Milestone,Temperature,Humidity,Soil_Type_clay,Soil_Type_loam,Soil_Type_sandy,Water_Frequency_bi-weekly,Water_Frequency_daily,Water_Frequency_weekly,Fertilizer_Type_chemical,Fertilizer_Type_none,Fertilizer_Type_organic
0,0.422172,1.3,-0.576016,-0.245965,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,-1.019510,5.2,-1.231412,0.150890,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [170]:
# Hacer predicciones con el modelo cargado
predictions = (modelo_cargado.predict(df) > 0.5).astype("int32")
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
[[0]
 [0]]
